# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
import json
import requests 
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database 'uk_food' was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [9]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('6413c535bfe512e97ff73a7a'), 'FHRSID': 1034540, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000078691', 'BusinessName': 'The Coastguard Inn', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'The Bay', 'AddressLine2': 'St Margarets Bay', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6DY', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2022-08-17T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'}, 'RightToReply': '', 'Distance': 4587.347174863443, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'totalP

In [10]:

# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [11]:
# Create a dictionary for the new restaurant data
item_to_add = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [12]:
# Insert the new restaurant into the collection
establishments.insert_one(item_to_add)

In [14]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName":item_to_add['BusinessName']}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6413c73550605ee10b72cd5e'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [34]:
# Another way to check that the new restaurant was inserted
# Filter results by name
query = {'BusinessName': 'Penang Flavours'}
results = establishments.find(query)
for result in results:
    pprint(result)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6413c73550605ee10b72cd5e'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [26]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Query
query = {'BusinessType': "Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID', 'BusinessType'}
sort = [('BusinessTypeID', 1)]
limit = 5

# Cast the results as a list and save them to a variable
bustype_results = list(establishments.find(query, fields).sort(sort).limit(limit))

# Pretty print the results
pprint(bustype_results)

[{'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6413c535bfe512e97ff73a7f')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6413c535bfe512e97ff73a84')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6413c535bfe512e97ff73a7b')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6413c535bfe512e97ff73a7d')},
 {'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('6413c535bfe512e97ff73a83')}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [27]:
# Update the new restaurant with the correct BusinessTypeID
db.establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set':
        {'BusinessTypeID': 1}
     }
)

In [31]:
# Confirm that the new restaurant was updated
updated_resto = db.establishments.find({'BusinessName': 'Penang Flavours'})

# See the change in collection.
for i in updated_resto:
    pprint(i)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6413c73550605ee10b72cd5e'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [39]:
# Find how many documents have LocalAuthorityName as "Dover"
query_delete = {'LocalAuthorityName': "Dover"}
fields = {'BusinessName', 'BusinessType', 'LocalAuthorityName'}
sort = [('BusinessName', 1)]
limit = 10

# Print the number of results
# There should be 994
print("Number of documents that have LocalAuthorityName as Dover :", establishments.count_documents(query_delete))
print("---------------------------------------")

# Cast the results as a list and save them to a variable
# This is just to double check that the query was for Dover LocalAuthority
dover_results = list(establishments.find(query_delete, fields).sort(sort).limit(limit))
# Pretty print the results
pprint(dover_results)

Number of documents in result: 994
[{'BusinessName': '4-5 Market Place',
  'BusinessType': 'Retailers - other',
  'LocalAuthorityName': 'Dover',
  '_id': ObjectId('6413c535bfe512e97ff744f9')},
 {'BusinessName': '81 Beach Street',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'LocalAuthorityName': 'Dover',
  '_id': ObjectId('6413c535bfe512e97ff73bf5')},
 {'BusinessName': 'A Baked Occasion',
  'BusinessType': 'Other catering premises',
  'LocalAuthorityName': 'Dover',
  '_id': ObjectId('6413c535bfe512e97ff73d18')},
 {'BusinessName': 'Abbeyfield House',
  'BusinessType': 'Caring Premises',
  'LocalAuthorityName': 'Dover',
  '_id': ObjectId('6413c535bfe512e97ff73ab3')},
 {'BusinessName': 'Able Catering',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'LocalAuthorityName': 'Dover',
  '_id': ObjectId('6413c535bfe512e97ff7423c')},
 {'BusinessName': 'Action Sports Nutrition LLP',
  'BusinessType': 'Retailers - other',
  'LocalAuthorityName': 'Dover',
  '_id': ObjectId('6413c535bfe512e97ff7466

In [40]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query_delete)
results_after_delete = establishments.find(query_delete)
for result in results_after_delete:
    print(result)

In [41]:
# Check if any remaining documents include Dover
# Results should be zero if all the documents were deleted correctly
print("Number of documents in result:", establishments.count_documents(query_delete))

Number of documents in result: 0


In [42]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6413c535bfe512e97ff73d60'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [43]:
# Change the data type from String to Decimal for longitude
# Change the data type from String to Decimal for latitude
# Change the data type from String to Double 
establishments.update_many({}, [ {'$set': {"geocode.longitude" : {'$toDouble': "$geocode.longitude"},
                                           "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}
                                } ]
                           )


In [44]:
# Check that the coordinates are now numbers
establishments.find_one()

{'_id': ObjectId('6413c535bfe512e97ff73d60'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00: